# data processing into a single dataframe


In [20]:
!pip install pandas
import pandas as pd
'''
Feature Set 1:
['aapos',
 'TSL',
 'codonpos',
 'HUVEC_fitCons_score',
 'ExAC_cnv.score',
 'GDI',
 'LoFtool_score',
 'SORVA_LOF_MAF0.005_HetOrHom',
 'Problematic']


for i in 2 3 4; do python3 keep_relevant_headers.py \
../data/chr2-4_with_gene/chr$i\_gene.tsv \
../data/chr2-4_with_gene/chr$i.out_set1.tsv \
--cols chr,pos,ref,alt,aapos,APPRIS,TSL,codonpos,Ancestral_allele,ExAC_cnv.score,\
HUVEC_fitCons_score,clinvar_clnsig,Interpro_domain,GDI,LoFtool_score,SORVA_LOF_MAF0.005_HetOrHom,\
Essential_gene_CRISPR; done

'''


df_all = pd.read_csv('../../chr2_out.tsv', sep='\t', encoding = "UTF-8")
df_all = df_all.set_index(['chr','pos', 'ref', 'alt']) #index by identifier

for c in range (3,4):
    #print(c)
    path = '../../chr' + str(c) + '_out.tsv'
    df = pd.read_csv(path, sep='\t', encoding = "UTF-8")
    df = df.set_index(['chr','pos', 'ref', 'alt']) #index by identifier
    df_all= df_all.append(df)
    
print('concat shape', df_all.shape)

df = df_all
print("shape before", df.shape)
df = df.replace({'.': 0})  ## FIXME: atm replace with nan and drop: later find a way to impute, maybe 0 but not sure
#print("SUM", pd.isnull(df).sum())  ## Use this to see how many elements have nan 
df = df.dropna()
print("shape after dropping all data with score = '.'", df.shape)
## still contains some mixed labels, see them by: df[~df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]

### prior to remapping of clinical significance labels to only pathogenic/benign, check out the distirbutions:
#print("TOTAL SUM OF clinsig: \n", df['clinvar_clnsig'].value_counts())
#print("PERCENT OF clinsig: \n", df['clinvar_clnsig'].value_counts(normalize=True))
#print("CLINVAR LABELS \n", df.groupby('clinvar_clnsig').count())


features_mapping = {
    'APPRIS': {
        'principal1': 5,
        'principal2': 4, 
        'principal3': 3, 
        'principal4': 2, 
        'principal5': 1,
        'alternative1': -1,
        'alternative2': -2
    },
    'Ancestral_allele': {
        'A': 2,
        'C': 2,
        'T': 2,
        'G': 2,
        'a': 1,
        'c': 1,
        't': 1,
        'g': 1,
        'N': 1,
        '-': -2,
    },
    'Essential_gene_CRISPR': {
        'E': -1,
        'N': 1
    }
}
## Map labels to 1/0 and remove Unkn
clinvar_labels={
    'Pathogenic': 1,
    'Likely_pathogenic': 1,
    'Pathogenic/Likely_pathogenic': 1,
    'Benign/Likely_benign': 0,
    'Likely_benign': 0,
    'Benign': 0
}

df = df.replace(features_mapping)
print(list(clinvar_labels.keys()))
print(df.head())
df = df[df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]  # best to drop everything that isn't in the dict
print(df.head())
df['Problematic'] = df['clinvar_clnsig'].map(clinvar_labels)

df= df.drop(columns=['clinvar_clnsig', 'Interpro_domain'])#, 'clinvar_id'])  #tbh, I don't know why I kept them. consider adding to index
print("shape after dropping Unk", df.shape)
print(df.head())
### pickle to reuse dataframe:
df.to_pickle("./pickled/chr2-4_featureset2.pkl")



#print("PROBLEMATIC LABEL DISTRIBUTION \n", df.groupby('Problematic').count())
print("TOTAL SUM OF LABELS: \n", df['Problematic'].value_counts())
print("PERCENT OF LABELS: \n", df['Problematic'].value_counts(normalize=True))

concat shape (38383, 13)
shape before (38383, 13)
shape after dropping all data with score = '.' (38244, 13)
['Pathogenic', 'Likely_pathogenic', 'Pathogenic/Likely_pathogenic', 'Benign/Likely_benign', 'Likely_benign', 'Benign']
                     aapos  APPRIS TSL codonpos  Ancestral_allele  \
chr pos     ref alt                                                 
2   272223  G   A      102       0   1        1                 2   
                A       50       0   2        2                 2   
    277003  A   G      106      -1   1        2                 2   
    1436306 C   A      135       0   1        2                 2   
    1436345 C   T      148       0   1        2                 2   

                    HUVEC_fitCons_score          clinvar_clnsig  \
chr pos     ref alt                                               
2   272223  G   A              0.714379  Uncertain_significance   
                A              0.714379  Uncertain_significance   
    277003  A   G   